# Phase 2: Historical Data Extraction via Web Scraping

**Data Source:** [Wikipedia - List of Falcon 9 and Falcon Heavy launches](https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches)

**Executive Summary:**
To ensure the model has a complete historical context, this notebook employs web scraping techniques to extract data from semi-structured web sources. By targeting Wikipedia’s launch tables, we supplement the API data with additional historical records, verifying our dataset's integrity against public documentation.

**Objectives:**
- Web scraping with BeautifulSoup
- HTML parsing and data extraction
- Data cleaning and transformation
- Creating structured datasets for analysis

**Acknowledgments:**
- Original lab structure from IBM Data Science Professional Certificate

## 1) Import Required Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 2) Helper functions to process web scraped HTML table

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

## 3) Fetch Wikipedia Page

In [3]:
# Using a static snapshot to ensure reproducibility
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Headers to mimic browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# Make HTTP GET request
response = requests.get(static_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    print(f" Successfully fetched page (Status: {response.status_code})")
else:
    print(f" Failed to fetch page (Status: {response.status_code})")

 Successfully fetched page (Status: 200)


## 4) Parse HTML with BeautifulSoup

In [4]:
# Create BeautifulSoup object
soup = BeautifulSoup(response.content, 'html.parser')

# Verify by checking page title
print(f"Page Title: {soup.title.string}")

Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


## 5) Extract Column Names from Table Headers

In [5]:
# Find all tables on the page
html_tables = soup.find_all('table', "wikitable plainrowheaders collapsible")

print(f"Total tables found: {len(html_tables)}")

# The third table (index 2) contains the first set of launch records
first_launch_table = html_tables[2]

# Extract column names from table headers
column_names = []

for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print(f"\nExtracted Columns ({len(column_names)}):")
print(column_names)

Total tables found: 9

Extracted Columns (8):
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## 6) Initialize Data Dictionary

In [6]:
# Create a dictionary with column names as keys
launch_dict = dict.fromkeys(column_names)

# Remove irrelevant column
del launch_dict['Date and time ( )']

# Initialize each column as an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Add additional parsed columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

## 7) Extract Launch Records from All Tables

In [7]:
extracted_row = 0

# Iterate through all launch tables
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    
    # Process each row in the table
    for rows in table.find_all("tr"):
        
        # Check if row contains a valid flight number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # Get all table data cells
        row = rows.find_all('td')
        
        # Process only rows with valid flight numbers
        if flag:
            extracted_row += 1
            
            # Flight Number
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            
            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else "Unknown"
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else "Unknown"
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else "Unknown"
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string if row[5].a else "Unknown"
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else "Unknown"
            launch_dict['Customer'].append(customer)
            
            # Launch Outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

print(f"\n Successfully extracted {extracted_row} launch records")


 Successfully extracted 121 launch records


## 8) Create DataFrame and Initial Inspection

In [8]:
# Convert dictionary to DataFrame
df = pd.DataFrame(launch_dict)

print(f"DataFrame Shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

DataFrame Shape: (121, 11)

First 5 rows:


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


In [9]:
# Check data types and missing values
print("Data Info:")
df.info()

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Flight No.       121 non-null    object
 1   Launch site      121 non-null    object
 2   Payload          121 non-null    object
 3   Payload mass     121 non-null    object
 4   Orbit            121 non-null    object
 5   Customer         120 non-null    object
 6   Launch outcome   121 non-null    object
 7   Version Booster  121 non-null    object
 8   Booster landing  121 non-null    object
 9   Date             121 non-null    object
 10  Time             121 non-null    object
dtypes: object(11)
memory usage: 10.5+ KB


In [10]:
# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())


Missing Values:
Flight No.         0
Launch site        0
Payload            0
Payload mass       0
Orbit              0
Customer           1
Launch outcome     0
Version Booster    0
Booster landing    0
Date               0
Time               0
dtype: int64


In [11]:
# Display sample of the data
print("\nSample Records:")
df.sample(10)


Sample Records:


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
18,19,Cape Canaveral,SpaceX CRS-7,"1,952 kg",LEO,NASA,Failure,F9 v1.1[,Precluded,28 June 2015,14:21
39,40,VAFB,Formosat-5,475 kg,SSO,NSPO,Success\n,F9 FT[,Success,24 August 2017,18:51
77,78,CCAFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5,Success,7 January 2020,02:19:21
102,103,KSC,NROL-108,C,LEO,NRO,Success\n,F9 B5 ♺,Success,19 December 2020,14:00:00
91,92,CCAFS,SAOCOM 1B,"3,130 kg",SSO,CONAE,Success\n,F9 B5,Success,30 August 2020,23:18
28,29,VAFB,Iridium NEXT,"9,600 kg",Polar,Iridium Communications,Success\n,F9 FT[,Success,14 January 2017,17:54
57,58,CCAFS,Telstar 19V,"7,075 kg",GTO,Telesat,Success,F9 B5,Success,22 July 2018,05:50
118,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1063.2665,Success,26 May 2021,18:59
34,35,KSC,SpaceX CRS-11,"2,708 kg",LEO,NASA,Success\n,F9 FT[,Success,3 June 2017,21:07
73,74,CCAFS,AMOS-17,"6,500 kg",GTO,Spacecom,Success\n,F9 B5B1047.3472,No attempt,6 August 2019,23:23


## 9) Data Quality Checks

In [12]:
# Check unique values in categorical columns
print("Unique Launch Sites:")
print(df['Launch site'].unique())

print("\nUnique Orbits:")
print(df['Orbit'].unique())

print("\nUnique Launch Outcomes:")
print(df['Launch outcome'].unique())

print("\nUnique Booster Landing Statuses:")
print(df['Booster landing'].unique())

Unique Launch Sites:
['CCAFS' 'VAFB' 'Cape Canaveral' 'KSC' 'CCSFS']

Unique Orbits:
['LEO' 'Polar orbit' 'GTO' 'HEO' 'Polar' 'SSO' 'MEO' 'Sub-orbital']

Unique Launch Outcomes:
['Success\n' 'Success' 'Failure']

Unique Booster Landing Statuses:
['Failure' 'No\xa0attempt\n' 'No attempt' 'Uncontrolled' 'No\xa0attempt'
 'Controlled' 'Failure ' 'Precluded' 'Success']


## 10) Export to CSV

In [13]:
# Export cleaned data to CSV
output_filename = 'spacex_web_scraped.csv'
df.to_csv(output_filename, index=False)

print(f" Data successfully exported to '{output_filename}'")
print(f"  Total records: {len(df)}")
print(f"  Total columns: {len(df.columns)}")

 Data successfully exported to 'spacex_web_scraped.csv'
  Total records: 121
  Total columns: 11
